In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

In [ ]:
google_apps = pd.read_csv('Google Play Store Apps.zip')
google_reviews = pd.read_csv('google_app_reviews.csv')

In [ ]:
google_apps.rename(columns={'App Id': 'app_id'}, inplace=True)

In [ ]:
apps = google_apps.merge(google_reviews, on="app_id", how='inner')

In [ ]:
apps.columns

In [ ]:
apps = apps.groupby('app_id').mean()

In [ ]:
apps = apps.copy()
apps.dropna(inplace=True)

apps['Rating'] = np.sqrt(apps['Rating'])

In [ ]:
apps['likes'] = apps['likes'] + 1
apps['polarity_weight'] = apps['likes'] * apps['polarity']

In [ ]:
apps['Editors Choice'].unique()

In [ ]:
apps.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
apps['installs'] = (apps['Minimum Installs'] + apps['Maximum Installs']) / 2
apps.drop(['Minimum Installs', 'Maximum Installs'], axis=1, inplace=True)

In [ ]:
apps.drop('score', axis=1, inplace=True)

In [ ]:
features = ['Rating Count', 'installs', 'polarity_weight']

In [ ]:
plt.rcParams.update({'xtick.labelsize': 'xx-large', 'ytick.labelsize': 'xx-large'})
plt.figure(figsize=(20, 11))
cor = apps.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

# Select features and target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(apps[features], apps['Rating'], random_state = 1)

# Decision Tree 🌲

In [ ]:
decision_tree = DecisionTreeRegressor(max_leaf_nodes=50, min_samples_split=100, min_samples_leaf=100, max_features=2, random_state = 1)
decision_tree.fit(X_train, y_train)

val_predictions = np.around(decision_tree.predict(X_test), decimals=2)
print("MAE: ",mean_absolute_error(y_test, val_predictions))
print("Decision Tree Score: ", decision_tree.score(X_test, y_test))

# Random Forest 🌳🌳🌳🌳

In [ ]:
forest_model = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(max_depth=15, n_estimators=100, min_samples_leaf=100, min_samples_split=500)
).fit(X_train, y_train)

preds = forest_model.predict(X_test)
print("MAE: ",mean_absolute_error(y_test, preds))
print("Random Forest Score: ", forest_model.score(X_test, y_test))

# Gradient Boosting Regressor

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100, min_samples_leaf=10, max_depth=50)
gbr.fit(X_train, y_train)

print("MAE: ",mean_absolute_error(y_test, gbr.predict(X_test)))
print("GBR Score: ", gbr.score(X_test, y_test))